In [1]:
import requests
import pandas as pd
import json
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path='../vplacement/config/.env')

True

In [2]:
def get_input_recipe(url):
    """ 
    Function to retrieve the information about the input recipe.
  
    Parameters: 
    url (str): URL of the recipe of interest
  
    Returns: 
    JSON: Recipe characteristics
  
    """
    base_url = "https://api.spoonacular.com/recipes/extract"
    parameters = {"apiKey": os.getenv('API_KEY'), 
                  "url":url}
    response = requests.request("GET", base_url, params=parameters)
    response_json = response.json()
    return response_json

In [3]:
#response_json = get_input_recipe("https://www.mynewroots.org/site/2014/01/big-comfy-sweet-potato/")
#response_json = get_input_recipe("https://www.beefitswhatsfordinner.com/recipes/recipe/55558/classic-beef-meatloaf")
# response_json = get_input_recipe("https://www.simplyrecipes.com/recipes/tandoori_chicken/")
# response_json = get_input_recipe("https://www.foodandwine.com/recipes/mushroom-and-chicken-risotto") :(
response_json = get_input_recipe("https://www.ricardocuisine.com/en/recipes/5762-classic-beef-chili")

In [4]:
response_json

{'vegetarian': False,
 'vegan': False,
 'glutenFree': False,
 'dairyFree': False,
 'veryHealthy': False,
 'cheap': False,
 'veryPopular': False,
 'sustainable': False,
 'weightWatcherSmartPoints': 0,
 'gaps': 'no',
 'lowFodmap': False,
 'ketogenic': False,
 'whole30': False,
 'preparationMinutes': 15,
 'cookingMinutes': 60,
 'sourceUrl': 'https://www.ricardocuisine.com/en/recipes/5762-classic-beef-chili',
 'aggregateLikes': 0,
 'spoonacularScore': 0.0,
 'healthScore': 0.0,
 'pricePerServing': 0.0,
 'extendedIngredients': [{'id': 11282,
   'aisle': 'Produce',
   'image': 'brown-onion.png',
   'consitency': 'solid',
   'name': 'onion',
   'original': '1 onion, finely chopped',
   'originalString': '1 onion, finely chopped',
   'originalName': 'onion, finely chopped',
   'amount': 1.0,
   'unit': '',
   'meta': ['finely chopped'],
   'metaInformation': ['finely chopped'],
   'measures': {'us': {'amount': 1.0, 'unitShort': '', 'unitLong': ''},
    'metric': {'amount': 1.0, 'unitShort': '',

In [5]:
def transform_to_ingredients(json):
    # Get ingredients
    ingredients = response_json['extendedIngredients']

    # Create dataframe
    df = pd.DataFrame(ingredients)
    # Exclude meat from dataframe
    df = df[df['aisle'] != 'Meat']
    df = df[-((df['name'].str.contains('meat')) |
              (df['name'].str.contains('beef')) |
              (df['name'].str.contains('chicken')))]

    ingredients_list = df['name'].tolist()

    ingredients_str = ','.join(ingredients_list)
    return ingredients_str

In [6]:
ingredients_str = transform_to_ingredients(response_json)

In [7]:
ingredients_str

'onion,jalapeno pepper,butter,garlic,chili powder,ground cumin,canned tomatoes,kidney beans,strong coffee,ketchup,lime juice,fresh cilantro'

In [8]:
def get_output_recipes(ingredients, number=20, ranking=1, ignore_pantry=True):
    """
    Function return output recipe based on original ingredients

    Parameters:
    ingredients (str): ingredients of the recipe

    Returns:
    JSON: Recipes
    """
    base_url = "https://api.spoonacular.com/recipes/findByIngredients"
    parameters = {"apiKey": os.getenv('API_KEY'),
                  "ingredients": ingredients,
                  "number": number,
                  "ranking": ranking,
                  "ignorePantry": ignore_pantry,
                  }
    response = requests.request("GET", base_url, params=parameters)
    
    output_recipes = response.json()
    return output_recipes

In [9]:
output_recipes = get_output_recipes(ingredients_str)

In [10]:
def remove_meat_recipes(output_recipes):
    result = []
    result_ids = []
    meat_words = ['chicken', 'beef', 'meat']
    output_recipes_veg = []
    for recipe in range(0, len(output_recipes)):
        for ingredient in range(0, len(output_recipes[recipe]['missedIngredients'])):
            # Only continue when the recipe is not already found to have meat
            if output_recipes[recipe]['id'] not in (result_ids):
                # Add recipe id to list if the recipe contains meat or seafood
                if ((output_recipes[recipe]['missedIngredients'][ingredient]['aisle'] == 'Meat') |
                        (output_recipes[recipe]['missedIngredients'][ingredient]['aisle'] == 'Seafood') |
                        any(word in output_recipes[recipe]['missedIngredients'][ingredient]['name']
                            for word in meat_words)):
                    result.append(output_recipes[recipe])
                    result_ids.append(output_recipes[recipe]['id'])
        if output_recipes[recipe]['id'] not in (result_ids):
            output_recipes_veg.append(output_recipes[recipe])
    #output_recipes_veg = json.dumps({"output_recipes": output_recipes_veg})
    #output_recipes_veg = json.loads(output_recipes_veg)
    return output_recipes_veg

In [25]:
len(remove_meat_recipes(output_recipes))

7

In [26]:
output_recipes_veg = remove_meat_recipes(output_recipes)

In [56]:
def remove_allergies(output_recipes_veg, allergies):
    result = []
    result_ids = []
    allergy_free_recipes = []
    allergy_list = allergies.split(', ')
    for recipe in range(0, len(output_recipes_veg)):
        for ingredient in range(0, len(output_recipes_veg[recipe]['missedIngredients'])):
            # Only continue when the recipe is not already found to have an allergy ingredient
            if output_recipes_veg[recipe]['id'] not in result_ids:
                # Add recipe id to list if the recipe contains an allergy ingredient
                if any(word in output_recipes_veg[recipe]['missedIngredients'][ingredient]['name'] for word in allergy_list):
                    result.append(output_recipes_veg[recipe])
                    result_ids.append(output_recipes_veg[recipe]['id'])
        if output_recipes_veg[recipe]['id'] not in result_ids:
            allergy_free_recipes.append(output_recipes_veg[recipe])
#     #allergy_free_recipes = json.dumps({"output_recipes": allergy_free_recipes})
#     #output_recipes_veg = json.loads(allergy_free_recipes)
    return allergy_free_recipes

In [57]:
len(remove_allergies(output_recipes_veg, 'tomatoes'))

1

In [58]:
allergy_list = 'tomato'.split(', ')

In [59]:
allergy_list

['tomato']

In [60]:
remove_allergies(output_recipes_veg, 'tomato')

[{'id': 145288,
  'title': 'My Salsa',
  'image': 'https://spoonacular.com/recipeImages/145288-312x231.jpg',
  'imageType': 'jpg',
  'usedIngredientCount': 7,
  'missedIngredientCount': 1,
  'missedIngredients': [{'id': 10511282,
    'amount': 0.5,
    'unit': 'cup',
    'unitLong': 'cups',
    'unitShort': 'cup',
    'aisle': 'Produce',
    'name': 'yellow onion',
    'original': '1/2 cup sweet yellow onion, roughly chopped',
    'originalString': '1/2 cup sweet yellow onion, roughly chopped',
    'originalName': 'sweet yellow onion, roughly chopped',
    'metaInformation': ['sweet', 'yellow', 'roughly chopped'],
    'extendedName': 'sweet yellow onion',
    'image': 'https://spoonacular.com/cdn/ingredients_100x100/brown-onion.png'}],
  'usedIngredients': [{'id': 10011693,
    'amount': 28.0,
    'unit': 'ounce',
    'unitLong': 'ounces',
    'unitShort': 'oz',
    'aisle': 'Canned and Jarred',
    'name': 'canned tomatoes',
    'original': '1 (28 ounce) can diced tomatoes',
    'orig